# LSTM을 이용한 텍스트 생성
- 데이터 다운로드: 캐글 new york times comments
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf
seed=2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
import pandas as pd
df = pd.read_csv('data/ArticlesApril2018.csv', encoding='latin1')
df.head(3)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...


## 데이터 전처리

In [4]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [5]:
# headline 열 데이터만 사용, null 검사
df.headline.isnull().sum()

0

In [6]:
headlines = [text for text in df.headline.values]
len(headlines)

1324

In [7]:
headlines[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
# 노이즈 데이터('Unknown') 제거
headlines = [text for text in headlines if text != 'Unknown']
len(headlines)

1214

In [9]:
# 구둣점 제거, 소문자로 변환
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def preprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [11]:
headlines = [preprocessing(s) for s in headlines]
headlines[:3]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained']

In [12]:
# 단어 집합을 만들고, 그 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
vocab_size

3494

In [13]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in headlines: 
    encoded = t.texts_to_sequences([line])[0]   #바깥쪽에 있는 리스트 제외
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
sequences[:11]
# [0,0,0,0,8,1]
# 00000->패딩(x를 0으로 채움) 8,1->y값

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [14]:
max_len = max(len(s) for s in sequences)
max_len

24

## Padding
- 전체 샘플의 길이를 가장 긴 샘플의 길이(6) 패딩
- 'pre' 옵션을 주면 앞을 0으로 채움

In [15]:
# 전체 샘플의 길이를 6(가장 긴 샘플의 길이)으로 패딩
# 'pre' 옵션을 주면 앞을 0으로 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115]])

In [16]:
X = sequences[:,:-1]
y = sequences[:,-1]
Y = to_categorical(y)
X.shape, Y.shape

((7803, 23), (7803, 3494))

## 모델 정의
- Embedding
- SimpleRNN

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [19]:
# 임베딩 백터 10차원, GRU 노드 크기 128
model = Sequential([
    Embedding(vocab_size, 10, input_length=max_len-1),
    GRU(128),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 23, 10)            34940     
_________________________________________________________________
gru (GRU)                    (None, 128)               53760     
_________________________________________________________________
dense (Dense)                (None, 3494)              450726    
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [21]:
hisotry = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.7057 - accuracy: 0.0291
Epoch 2/200
244/244 - 3s - loss: 7.1229 - accuracy: 0.0315
Epoch 3/200
244/244 - 3s - loss: 6.9182 - accuracy: 0.0432
Epoch 4/200
244/244 - 3s - loss: 6.7109 - accuracy: 0.0495
Epoch 5/200
244/244 - 3s - loss: 6.4868 - accuracy: 0.0534
Epoch 6/200
244/244 - 3s - loss: 6.2371 - accuracy: 0.0556
Epoch 7/200
244/244 - 3s - loss: 5.9657 - accuracy: 0.0624
Epoch 8/200
244/244 - 3s - loss: 5.6905 - accuracy: 0.0679
Epoch 9/200
244/244 - 3s - loss: 5.4226 - accuracy: 0.0759
Epoch 10/200
244/244 - 3s - loss: 5.1652 - accuracy: 0.0834
Epoch 11/200
244/244 - 4s - loss: 4.9260 - accuracy: 0.0950
Epoch 12/200
244/244 - 4s - loss: 4.6989 - accuracy: 0.1152
Epoch 13/200
244/244 - 3s - loss: 4.4823 - accuracy: 0.1402
Epoch 14/200
244/244 - 3s - loss: 4.2784 - accuracy: 0.1656
Epoch 15/200
244/244 - 3s - loss: 4.0816 - accuracy: 0.2013
Epoch 16/200
244/244 - 3s - loss: 3.8975 - accuracy: 0.2300
Epoch 17/200
244/244 - 3s - loss: 3.7177 - accura

## 모델 검증

In [32]:
from my_util import sentence_generation

In [33]:
print(sentence_generation(model, t, max_len, 'i', 10))
print(sentence_generation(model, t, max_len, 'epa', 10))
print(sentence_generation(model, t, max_len, 'former', 10))

TypeError: sentence_generation() takes 4 positional arguments but 5 were given